#  The Static and Shared Library with GCC



##  The Library 

Library files are a collection of **precompiled functions** that are written according to the principle of reusability. 

They are usually made up of a set of interrelated functions used to perform a common task

### Static  Library 

Static libraries are **directly linked into** the program at compile time. 

A program using a static library takes copies of the code that it uses from the static library and makes it part of the program. 



### Shared  Library 

For a shared library, the compiler/linker checks that the names you link with exist in the library when the application is built, **but doesn't move their code into the application**. At run time, the shared library must be available.

Before the executable starts running, **the operating system loads the machine code needed for the external functions** - a process known as **dynamic linking.** 

### The advantages and disadvantages in each method

**The advantages of using library**

1. Modular development

2. Reusability

3. Maintainability

**Shared libraries**

* reduce the amount of code that is duplicated in each program that makes use of the library, keeping **binaries small**. 

* allows you to replace the shared object with one that is functionally equivalent, but may have added performance benefits **without needing to recompile** the program that makes use of it. 

Shared libraries will, however have a small additional cost for the execution of the functions as well as a run-time loading cost as all the symbols in the library need to be connected to the things they use

**Static libraries**

* increase the overall size of the binary, but it means that you don't need to carry along a copy of the library that is being used. As the code is connected at compile time there are not any additional run-time loading costs. The code is simply there.

![library](./img/library.png) 




**GCC assumes that all libraries**
   
* `start` with **lib**

* Static libraries: `end` with `.a`

* Shared libraries: `end`  with `.dll`(windows) or `.so`(Linux)，



Change the current working directory to `./demo`

In [2]:
%cd demo

J:\SEU\SEE\PySEE\home\notebook\demo


In [1]:
%pwd

'J:\\SEU\\SEE\\PySEE\\home\\notebook'


## 1 The shared library

### 1.1 The source code of library 
The  shared library we will build consist of a single source file: 

* `statistics.c/h`


In [2]:
%%file ./include/statistics.h
#ifndef STATISTICS_H
#define STATISTICS_H

double  mean(double data[], int size);

#endif


Overwriting ./include/statistics.h


In [3]:
%%file ./src/statistics.c

#include "statistics.h"

double  mean(double data[], int size)
{
  /* Compute the arithmetic mean of a dataset using the recurrence relation 
     mean_(n) = mean(n-1) + (data[n] - mean(n-1))/(n+1)   */

  double mean = 0;
  for(int  i = 0; i < size; i++)
  {
      mean += (data[i] - mean) / (i + 1);
   }
  return mean;
}

Overwriting ./src/statistics.c



### 1.2 The shared library



#### Step 1: Compiling with Position Independent Code

We compile the C file with `Position Independent Code( PIC )` 


* **`-fPIC`** : stands for `Position Independent Code`(位置无关代码)
   
   the generated machine code is `not dependent` on being located at a `specific address` in order to `work`.
   
   Position-independent code can be `executed` at `any memory address`
     

In [9]:
!gcc -c -O3 -Wall -fPIC  -o  ./obj/statistics.o  ./src/statistics.c -I./include/

* `-c`: compile into object file with default name : *.o.

      By default, the object file has the same name as the source file with extension of ".o" 
  
  
* `-O3`: Optimize yet more.

      turns on all optimizations specified by -O2 and also turns on the -finline-functions, -fweb, -frename-registers and -funswitch-loops optionsturns  on  all  optimizations   
     
  
* `-Wall`: prints "all"  compiler's warning message. 

      This option should always be used, in order to generate better code.

#### Step 2: Creating a shared library from an object file

* -shared : creating a shared library

In [10]:
!gcc -shared -o  ./bin/libstat.dll ./obj/statistics.o

**Makefile of Building the shared library**

In [12]:
%%file ./makestatso.mk

CC=gcc
CFLAGS=-O3 -Wall -fPIC 

SRC=./src/
OBJ=./obj/
BIN=./bin/
INC=./include/

# Windows
LIB=libstat.dll

all:  $(LIB)

$(LIB): statobj
	 $(CC) -shared -o $(BIN)$@ $(OBJ)statistics.o
    
statobj: $(SRC)statistics.c
	 $(CC) -c $(CFLAGS)  -o $(OBJ)statistics.o $(SRC)statistics.c -I$(INC)    


Overwriting ./makestatso.mk


In [13]:
!make -f makestatso.mk

gcc -c -O3 -Wall -fPIC   -o ./obj/statistics.o ./src/statistics.c -I./include/    
gcc -shared -o ./bin/libstat.dll ./obj/statistics.o


In [14]:
!dir .\bin\libstat*.*

 驱动器 J 中的卷是 cmh
 卷的序列号是 9C25-3306

 J:\SEU\SEE\PySEE\home\notebook\demo\bin 的目录

2021/04/13  20:07           309,891 libstat.dll
               1 个文件        309,891 字节
               0 个目录 92,349,657,088 可用字节


#### Step3: Using the shared library

##### Header Files and Libraries 

* `Header File`: When compiling the program, the **compiler** needs the **header** files to compile the source codes;

* `libraries`: the **linker** needs the **libraries** to resolve external references from other object files or libraries. 

The `compiler` and `linker` will not find the `headers/libraries` unless you set **the appropriate options**

* **1 Searching for Header Files**

   **`-Idir`:** The include-paths are specified via **-Idir** option (`uppercase` 'I' followed by the directory path. 
   
   
* **2 Searching for libraries Files**

   **`-Ldir`**: The library-path is specified via **-Ldir** option (`uppercase` 'L' followed by the directory path）. 


* **3 Linking the library**

   **`-lname`**: Link with the library name **without** the `lib` prefix and the extensions.
 


We use the **same** code ` ./src/statdemo.c` to demonstrates calling the shared library 



**Makefile**

In [15]:
%%file ./makestatsodemo.mk

SRC= ./src/
OBJ= ./obj/
BIN= ./bin/
INC=./include/

all: statdemoso

statdemoso: statobj  
	gcc -o $(BIN)$@ $(OBJ)statdemo.o -L$(BIN) -lstat

statobj: $(SRC)statdemo.c 
	gcc -c -o $(OBJ)statdemo.o $(SRC)statdemo.c -I$(INC)

Writing ./makestatsodemo.mk


In [16]:
!make -f makestatsodemo.mk

gcc -c -o ./obj/statdemo.o ./src/statdemo.c -I./include/
gcc -o ./bin/statdemoso ./obj/statdemo.o -L./bin/ -lstat


In [17]:
!dir .\bin\statdemo*.*

 驱动器 J 中的卷是 cmh
 卷的序列号是 9C25-3306

 J:\SEU\SEE\PySEE\home\notebook\demo\bin 的目录

2021/04/13  20:07           330,674 statdemoso.exe
               1 个文件        330,674 字节
               0 个目录 92,349,321,216 可用字节


In [18]:
!.\bin\statdemoso

mean is 4.400000


## 2 Building  a Shared Library with Multi-source

### 2.1  The multi-source files
The shared library we will build consist of the multi-source files

* statistics.c/h

* number.c/h


In [3]:
%%file ./include/number.h

#ifndef NUMBER_H
#define NUMBER_H


#ifdef __cplusplus
extern "C"
{
#endif
int factorial(int n);


#ifdef __cplusplus
}
#endif

#endif

Writing ./include/number.h


In [4]:
%%file ./src/number.c

#include "number.h"

// The factorial of a positive integer n, denoted by n!, 
//    is the product of all positive integers less than or equal to n. 
//  For example,5!=5*4*3*2*1=120
//  The value of 0! is 1 
int factorial(int n)
{
    if (n == 0 ) {
        return 1;
    }
    else 
    {
        return n * factorial(n - 1);
    }
}


Writing ./src/number.c


### 2.2 Building  into One Shared Library

**Makefile**

```make
SRCS=$(SRC)statistics.c \
    $(SRC)bubblesort.c 
```    

In [5]:
%%file ./makelibmaths.mk

CC=gcc
CFLAGS=-O3 -Wall -fPIC

SRC= ./src/
OBJ= ./obj/
BIN= ./bin/
INC= ./include/

LIB=libmaths.dll

SRCS=$(SRC)statistics.c \
    $(SRC)number.c 

# non-path filename
filename=$(notdir $(SRCS))

# the obj target of a source code using the pattern rule
OBJS=$(patsubst %.c,$(OBJ)%.o,$(filename))

all:$(LIB)
    
$(LIB): $(OBJS)  
	$(CC) -shared -o $(BIN)$@ $^ 

# the pattern rule: one step rule for multiple source files
$(OBJS):$(SRCS)
	$(CC) $(CFLAGS) -o $@ -c $(SRC)$(patsubst  %.o,%.c,$(notdir $@))  -I$(INC)

Overwriting ./makelibmaths.mk


In [6]:
!make -f makelibmaths.mk

gcc -O3 -Wall -fPIC -o obj/statistics.o -c ./src/statistics.c  -I./include/
gcc -O3 -Wall -fPIC -o obj/number.o -c ./src/number.c  -I./include/
gcc -shared -o ./bin/libmaths.dll obj/statistics.o obj/number.o 


In [7]:
!dir .\bin\libm*.dll

 驱动器 J 中的卷是 cmh
 卷的序列号是 9C25-3306

 J:\SEU\SEE\PySEE\home\notebook\demo\bin 的目录

2021/04/13  20:38           310,189 libmaths.dll
               1 个文件        310,189 字节
               0 个目录 92,344,991,744 可用字节


### 2.3 Building a client executable 

The code demonstrates calling the shared library's functions:


In [8]:
%%file ./src/demomath.c

#include <stdio.h> 
#include "statistics.h"
#include "number.h"

int main() {
    
     double a[] = {8, 4, 5, 3, 2};
     int len= sizeof(a)/sizeof(*a);
     printf("mena is %f\n", mean(a,  len)); 
    
     int n =5;
     printf("the factorial of  %d is %d\n",n,factorial(n));  // 5!=120
     return 0;
}

Overwriting ./src/demomath.c


In [9]:
!gcc -o  ./bin/demomath ./src/demomath.c  -L./bin/ -lmaths -I./include

In [10]:
!.\bin\demomath

mena is 4.400000
the factorial of  5 is 120


## 3 The C/C++ Caller of CoolProp Library in Windows

**Change working dir to `./notebook/coolpropdemo`**


In [ ]:
%pwd

In [53]:
%cd ../coolpropdemo

J:\SEU\SEE\PySEE\home\notebook\coolpropdemo



* [Official Doc: CoolProp Shared Library](http://www.coolprop.org/coolprop/wrappers/SharedLibrary/index.html#)

The example use a TDM-GCC-64 dll: **libCoolProp.dll** of CoolProp

**The Windows Shared Library**

1. The official Windows Shared Library: `CoolProp_x64.dll`

2. The Shared Library with MinGW64: `libCoolProp.dll`



#### C Caller

* -DCOOLPROP_LIB 

In [54]:
%%file ./main.c

/* 
The example use the shared library of CoolProp in c

The library: libCoolProp.dll or CoolProp_x64.dll

g++ -DCOOLPROP_LIB -o ./bin/main   main.c -I./include -L./bin -lCoolProp
       
./bin/main

*/

#include "CoolPropLib.h"
#include <stdio.h>

int main() {
     double value=PropsSI("P","T",273.15+0,"Q",0,"R134a");
     printf("%f\n", value); 
     return 0;
}


Overwriting ./main.c


Compiling

In [55]:
!g++  -DCOOLPROP_LIB -o ./bin/main  main.c -I./include -L./bin -lCoolProp

Run

In [56]:
!.\bin\main

292803.182339


#### C++ Caller

In [57]:
%%file ./demo.cpp
/* 
The example use the shared library of CoolProp in c

The library: libCoolProp.dll or CoolProp_x64.dll

g++ -DCOOLPROP_LIB -o ./bin/demo  demo.cpp -I./include -L./bin -lCoolProp

./bin/demo

*/

#include "CoolPropLib.h"
#include <iostream>
#include <iomanip>

int main()
{
    double value=PropsSI("P","T",273.15+0,"Q",0,"R134a");
    std::cout << std::setiosflags(std::ios::fixed)  << value << std::endl;
    return 1;
}

Overwriting ./demo.cpp


Compiling

In [58]:
!g++ -DCOOLPROP_LIB -o ./bin/demo  demo.cpp -I./include -L./bin -lCoolProp

Run

In [59]:
!.\bin\demo

292803.182339
